In [1]:
from scripts.make_dataset import prepare_data, stratified_user_split, merge_recipes_interactions, prepare_ffm_data, prepare_wd_data
from scripts.model import ffm_train_model, ffm_train_model_hyperparam_tuning, wd_train_model, naive_evaluation, ffm_predict, wd_predict, get_top_k_recommendations_existing_user
import pandas as pd

In [2]:
# A. Get dataset
print("\nA. Get dataset")

# Step A1. Prepare data and generate features
print("\nStep A1. Prepare data and generate features")
recipes, recipes_unscaled, interactions, user_enc, item_enc, scaler = prepare_data()


A. Get dataset

Step A1. Prepare data and generate features
Before downsampling: 1003724 interactions from 180310 users.
After downsampling: 97055 interactions from 18031 users.
Loaded 31319 ingredient embeddings from food2vec model, with dimension 100
Total Unique ingredients: 8517, Frequent ingredients (>= 100 occurrences): 507
Total unique fallback tags: 385
Total unique search terms: 93
Total unique tags: 498
Total unique final tags: 919
Train KMeans Clustering to identify clustering in food recipes:


100%|██████████| 42785/42785 [00:00<00:00, 48005.81it/s]


cluster
0     5143
1      122
2      377
3      246
4     7047
5     4735
6      498
7      228
8      840
9     5239
10     158
11    2068
12    7819
13    8147
14     118
Name: count, dtype: int64

Data preparation done


In [3]:
# Step A2. Stratified user split into train, validation, and test set
print("\nStep A2. Stratified user split into train, validation, and test set")
train_df, valid_df, test_df = stratified_user_split(interactions, recipes, valid_frac=0.15, test_frac=0.15)

train_merged, valid_merged, test_merged = merge_recipes_interactions(recipes, train_df, valid_df, test_df)


Step A2. Stratified user split into train, validation, and test set
Train users: 18031
Valid users: 1544
Test users: 1544
Data size before augmentation with negative samples:
Train size: 76237, Valid size: 10409, Test size: 10409

Generating negative samples for the training set with n_neg = 5:


100%|██████████| 76237/76237 [01:45<00:00, 722.23it/s] 
[Parallel(n_jobs=-1)]: Done 75989 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 76237 out of 76237 | elapsed:  1.8min finished



Generating negative samples for the validation set with n_neg = 10:


100%|██████████| 10409/10409 [00:24<00:00, 421.87it/s]
[Parallel(n_jobs=-1)]: Done 10409 out of 10409 | elapsed:   25.2s finished



Generating negative samples for the test set with n_neg = 20:


100%|██████████| 10409/10409 [00:36<00:00, 285.14it/s]
[Parallel(n_jobs=-1)]: Done 10366 out of 10409 | elapsed:   36.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 10409 out of 10409 | elapsed:   36.7s finished



Stratified user split into train, validation, and test set done

Recipe-interaction merge done


In [4]:
# Step A3. FFM: preprocess train data
print("\nStep A3. FFM: preprocess train data")
train_ffm_df, valid_ffm_df, test_ffm_df = prepare_ffm_data(train_merged, valid_merged, test_merged)


Step A3. FFM: preprocess train data

Writing ffm_train.txt
Progress: 0.0% (1/442,272 rows)
Progress: 20.0% (88,455/442,272 rows)
Progress: 40.0% (176,909/442,272 rows)
Progress: 60.0% (265,363/442,272 rows)
Progress: 80.0% (353,817/442,272 rows)
Progress: 100.0% (442,271/442,272 rows)
Progress: 100.0% (442,272/442,272 rows)
FFM file write complete.

Writing ffm_valid.txt
Progress: 0.0% (1/112,587 rows)
Progress: 20.0% (22,518/112,587 rows)
Progress: 40.0% (45,035/112,587 rows)
Progress: 60.0% (67,552/112,587 rows)
Progress: 80.0% (90,069/112,587 rows)
Progress: 100.0% (112,586/112,587 rows)
Progress: 100.0% (112,587/112,587 rows)
FFM file write complete.

Writing ffm_test.txt
Progress: 0.0% (1/216,768 rows)
Progress: 20.0% (43,354/216,768 rows)
Progress: 40.0% (86,707/216,768 rows)
Progress: 60.0% (130,060/216,768 rows)
Progress: 80.0% (173,413/216,768 rows)
Progress: 100.0% (216,766/216,768 rows)
Progress: 100.0% (216,768/216,768 rows)
FFM file write complete.

FFM data preparation d

In [5]:
# Step A4. WD: preprocess train data
print("\nStep A4. WD: preprocess train data")
tab_preprocessor, wide_preprocessor, X_tab_train, X_wide_train, y_train, X_tab_valid, X_wide_valid, y_valid, X_tab_test, X_wide_test, y_test = prepare_wd_data(train_ffm_df, valid_ffm_df, test_ffm_df)


Step A4. WD: preprocess train data


c:\Users\beung-yoga\Documents\GitHub4\.venv\lib\site-packages\pytorch_widedeep\preprocessing\tab_preprocessor.py:364: UserWarning: Continuous columns will not be normalised
  warnings.warn("Continuous columns will not be normalised")



WD data preparation done


In [6]:
# B. Train FFM and WD models
print("\nB. Train FFM and WD models")

# Step B1: Train FFM model
print("\nStep B1: Train FFM model")
ffm_model, test_predictions, eval_results = ffm_train_model(test_merged)


B. Train FFM and WD models

Step B1: Train FFM model
Training the model...
Training time: 520.6155
AUC: 0.5363
MAP@5:   0.1048
Recall@5: 0.2181
NDCG@5:   0.1459
MAP@10:   0.1286
Recall@10: 0.3960
NDCG@10:   0.2066
MAP@20:   0.1476
Recall@20: 0.6173
NDCG@20:   0.2717


In [5]:
# Step B1: Train FFM model
print("\nStep B1: FFM model hyperparameter finetuning")

# Specify a set of hyperparameters
lr_values = [0.01, 0.05, 0.1]
lambda_values = [0.0001, 0.0005, 0.001]
k_values = [8] # 8, 16, 32
opt_values = ["adagrad"] # "adagrad", "sgd", "ftrl"
epoch_values = [15] # 15, 30

param_dict = {"task": ["binary"], "metric": ["auc"], "lr": lr_values, "lambda": lambda_values, "k": k_values, "opt": opt_values, "epoch": epoch_values}

best_ffm_model, best_test_predictions, best_eval_result, tuned_results = ffm_train_model_hyperparam_tuning(test_merged, param_dict)


Step B1: FFM model hyperparameter finetuning

Parameter Set 1/9
Training FFM with: {'epoch': 15, 'k': 8, 'lambda': 0.0001, 'lr': 0.01, 'metric': 'auc', 'opt': 'adagrad', 'task': 'binary'}
Model trained successfully
AUC: 0.5366
MAP@5:   0.0998
Recall@5: 0.2235
NDCG@5:   0.1433
MAP@10:   0.1239
Recall@10: 0.4033
NDCG@10:   0.2052
MAP@20:   0.1433
Recall@20: 0.6297
NDCG@20:   0.2713
New best model found with MAP@10: 0.123935
Training time: 516.0627


c:\Users\beung-yoga\Documents\GitHub4\food-recommender\scripts\model.py:187: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tuned_results = pd.concat([



Parameter Set 2/9
Training FFM with: {'epoch': 15, 'k': 8, 'lambda': 0.0001, 'lr': 0.05, 'metric': 'auc', 'opt': 'adagrad', 'task': 'binary'}
Model trained successfully
AUC: 0.5326
MAP@5:   0.1006
Recall@5: 0.2249
NDCG@5:   0.1443
MAP@10:   0.1246
Recall@10: 0.4039
NDCG@10:   0.2058
MAP@20:   0.1440
Recall@20: 0.6287
NDCG@20:   0.2717
New best model found with MAP@10: 0.124571
Training time: 101.7585

Parameter Set 3/9
Training FFM with: {'epoch': 15, 'k': 8, 'lambda': 0.0001, 'lr': 0.1, 'metric': 'auc', 'opt': 'adagrad', 'task': 'binary'}
Model trained successfully
AUC: 0.5314
MAP@5:   0.1004
Recall@5: 0.2240
NDCG@5:   0.1441
MAP@10:   0.1244
Recall@10: 0.4024
NDCG@10:   0.2054
MAP@20:   0.1439
Recall@20: 0.6287
NDCG@20:   0.2716
Training time: 102.1127

Clearing memory after 3 iterations...
Reloaded test_merged from disk to free memory.
Reloaded test_merged from disk to free memory.

Parameter Set 4/9
Training FFM with: {'epoch': 15, 'k': 8, 'lambda': 0.0005, 'lr': 0.01, 'metric': '

In [6]:
# Step B2: Train WD model
print("\nStep B2: Train WD model")
wd_model, test_predictions, eval_results = wd_train_model(
    tab_preprocessor, wide_preprocessor, 
    X_tab_train, X_wide_train, y_train, X_tab_valid, X_wide_valid, y_valid, X_tab_test, X_wide_test, y_test, 
    test_merged
)


Step B2: Train WD model
Model structure:
WideDeep(
  (wide): Wide(
    (wide_linear): Embedding(60817, 1, padding_idx=0)
  )
  (deeptabular): Sequential(
    (0): TabMlp(
      (cont_norm): Identity()
      (encoder): MLP(
        (mlp): Sequential(
          (dense_layer_0): Sequential(
            (0): Linear(in_features=216, out_features=256, bias=True)
            (1): ReLU(inplace=True)
            (2): Dropout(p=0.1, inplace=False)
          )
          (dense_layer_1): Sequential(
            (0): Linear(in_features=256, out_features=128, bias=True)
            (1): ReLU(inplace=True)
            (2): Dropout(p=0.1, inplace=False)
          )
          (dense_layer_2): Sequential(
            (0): Linear(in_features=128, out_features=64, bias=True)
            (1): ReLU(inplace=True)
            (2): Dropout(p=0.1, inplace=False)
          )
          (dense_layer_3): Sequential(
            (0): Linear(in_features=64, out_features=32, bias=True)
            (1): ReLU(inplace=T

valid: 100%|██████████| 3519/3519 [00:31<00:00, 110.64it/s, loss=0.869]


AUC: 0.5345
MAP@5:   0.2500
Recall@5: 0.3192
NDCG@5:   0.3077
MAP@10:   0.2543
Recall@10: 0.4269
NDCG@10:   0.3349
MAP@20:   0.2662
Recall@20: 0.6211
NDCG@20:   0.3850


In [3]:
# Step B3: Evaluate naive model
print("\nStep B3: Evaluate naive model")
test_predictions, eval_results = naive_evaluation(interactions, test_merged)


Step B3: Evaluate naive model
AUC: 0.5053
MAP@5:   0.5881
Recall@5: 0.5686
NDCG@5:   0.6525
MAP@10:   0.5715
Recall@10: 0.5686
NDCG@10:   0.6339
MAP@20:   0.5674
Recall@20: 0.5686
NDCG@20:   0.6280


In [9]:
# Get user with least number of interaction, for testing
rare_user_id_enc = test_df["user_id_enc"].value_counts().idxmin()
pd.set_option("display.max_columns", None)

print("Get top food recommendations for the existing user, using FFM Model")
top_k_rec_ffm = get_top_k_recommendations_existing_user(
    rare_user_id_enc,
    user_enc,
    item_enc,
    interactions,
    recipes,
    predict_fn = ffm_predict,
    model = ffm_model,
    preprocessor_tab=None,
    preprocessor_wide=None,
    scaler=scaler,
    k=200
)

# Display results
top_k_rec_ffm.head(20)

Get top food recommendations for the existing user, using FFM Model
Progress: 0.0% (1/42,778 rows)
Progress: 20.0% (8,556/42,778 rows)
Progress: 40.0% (17,111/42,778 rows)
Progress: 60.0% (25,666/42,778 rows)
Progress: 80.0% (34,221/42,778 rows)
Progress: 100.0% (42,776/42,778 rows)
Progress: 100.0% (42,778/42,778 rows)
FFM file write complete.


,user_id,user_id_enc,recipe_id,item_id_enc,name,score,minutes,kcal,fat,sugar,sodium,protein,carb,cluster,techniques,final_tag_embeddings,ingredient_embeddings,cluster_vector
0,2000390617,14985,49,1,chicken breasts lombardi,0.271724,75.0,104.616667,6.333333,1.333333,5.833333,19.166667,0.666667,5,"[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...","[0.030205548, 0.22779678, -0.15320495, 0.38288...","[1.3954325, 1.0706503, 1.7713985, 2.1463203, 0...","[-0.16673428, -0.17127492, -0.19695784, -0.088..."
1,2000390617,14985,58,3,low fat burgundy beef vegetable stew,0.271716,164.0,46.683333,1.500000,6.000000,4.000000,9.833333,1.166667,9,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.30292073, 0.10292001, 0.010660671, 0.433282...","[-0.22141162, 1.0444584, 2.0060472, 0.95918703...","[-0.25116885, -0.25222975, -0.17730078, -0.111..."
2,2000390617,14985,66,5,black coffee barbecue sauce,0.271693,30.0,772.000000,6.000000,657.000000,93.000000,13.000000,63.000000,12,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0.24503809, 0.29496714, 0.01763307, 0.5456841...","[0.3610016, -1.0869095, 0.55841017, 0.2827703,...","[0.80593926, -0.17685801, 2.5648577, 0.989457,..."
3,2000390617,14985,62,4,black bean corn and tomato salad,0.271692,25.0,203.900000,11.500000,8.500000,0.000000,17.000000,9.000000,12,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.29056424, 0.14811099, 0.005352595, 0.491561...","[0.22162592, -0.031849474, 1.085998, -0.628254...","[-0.022034418, -0.08473698, -0.16677022, -0.16..."
4,2000390617,14985,91,7,brown rice and vegetable pilaf,0.271672,150.0,68.766667,6.166667,1.666667,2.833333,2.666667,2.333333,0,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[-0.060259607, 0.041048847, 0.14219505, 0.4718...","[0.005443096, 0.097978085, 1.6886278, 0.089179...","[-0.21898362, -0.17406645, -0.19555376, -0.125..."
5,2000390617,14985,109,11,butterflied lamb with garlic butter,0.271656,300.0,298.650000,28.750000,7.500000,3.500000,37.750000,4.250000,11,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0.25844678, 0.1406803, -0.15936458, 0.4366451...","[-0.12888122, 0.92462415, 1.9855738, 0.8978678...","[0.116058365, 0.20418806, -0.17098245, -0.1175..."
6,2000390617,14985,114,12,chicken breasts saltimbocca,0.271656,420.0,85.316667,7.500000,0.500000,9.333333,13.500000,0.833333,9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[-0.0073746014, 0.2610136, -0.2552812, 0.34653...","[1.2276641, 1.0024613, 1.745273, 1.8053932, 0....","[-0.19486293, -0.15173408, -0.20046803, -0.045..."
7,2000390617,14985,124,14,catherine s excellent yorkshire pudding,0.271652,35.0,35.400000,0.750000,0.000000,0.750000,2.750000,1.750000,13,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.15595397, 0.041491885, -0.023583915, 0.3829...","[2.2189775, 1.1844302, 1.7080477, 1.914067, 1....","[-0.26761365, -0.26479173, -0.20257413, -0.151..."
8,2000390617,14985,94,10,blueberry buttertarts,0.271651,40.0,28.158333,2.333333,3.833333,0.416667,0.666667,1.083333,13,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.11111164, 0.054656874, -0.09464598, 0.42955...","[0.040222242, -0.15441252, 2.1651247, -0.88449...","[-0.278168, -0.23827203, -0.18642728, -0.15571..."
9,2000390617,14985,92,8,brown bag apple salad,0.271650,10.0,35.350000,1.750000,20.500000,0.000000,1.000000,2.000000,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.45527613, 0.13363668, -0.06294151, 0.350360...","[-0.44718698, 0.04251325, 0.89771867, 0.83708,...","[-0.26768655, -0.24804243, -0.11622352, -0.160..."


In [10]:
# Get user with least number of interaction, for testing
rare_user_id_enc = test_df["user_id_enc"].value_counts().idxmin()
pd.set_option("display.max_columns", None)

print("Get top food recommendations for the existing user, using WD Model")
top_k_rec_wd = get_top_k_recommendations_existing_user(
    rare_user_id_enc,
    user_enc,
    item_enc,
    interactions,
    recipes,
    predict_fn = wd_predict,
    model = wd_model,
    preprocessor_tab = tab_preprocessor,
    preprocessor_wide = wide_preprocessor,
    scaler=scaler,
    k=200
)

# Display results
top_k_rec_wd.head(20)

Get top food recommendations for the existing user, using WD Model


,user_id,user_id_enc,recipe_id,item_id_enc,name,score,minutes,kcal,fat,sugar,sodium,protein,carb,cluster,techniques,final_tag_embeddings,ingredient_embeddings,cluster_vector
0,2000390617,14985,2886,257,best banana bread,0.970566,65.0,27.280000,1.600000,9.700000,1.400000,0.700000,1.400000,13,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...","[-0.05038603, 0.13221169, -0.05537756, 0.35124...","[1.5278765, 0.32666078, 1.2365386, 0.96930236,...","[-0.2794481, -0.25055483, -0.16171555, -0.1435..."
1,2000390617,14985,39087,7224,creamy cajun chicken pasta,0.969494,25.0,359.550000,31.500000,6.000000,17.000000,39.000000,7.500000,8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.009672228, 0.33944854, -0.06357586, 0.45481...","[1.8668798, 1.0509276, 1.3314042, 1.2940573, 1...","[0.20481667, 0.25024858, -0.17730078, 0.049406..."
2,2000390617,14985,32204,5908,whatever floats your boat brownies,0.966407,35.0,24.418750,1.875000,10.062500,0.437500,0.750000,1.062500,13,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0.22047296, -0.04438333, -0.1618363, 0.389596...","[1.8044553, 0.22999617, 1.2127188, 1.5195085, ...","[-0.2836182, -0.24594878, -0.16018863, -0.1554..."
3,2000390617,14985,99476,18173,the best easy beef and broccoli stir fry,0.956828,25.0,37.525000,2.500000,7.750000,7.500000,1.750000,1.500000,13,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0.095521554, 0.109438516, -0.002647612, 0.587...","[1.2459449, -0.189031, 1.5333778, 2.9894283, 1...","[-0.2645166, -0.23548049, -0.1699294, -0.06810..."
4,2000390617,14985,89204,16321,crock pot chicken with black beans cream cheese,0.953545,243.0,169.800000,13.250000,9.750000,9.500000,22.750000,4.000000,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[-0.008186579, 0.2077937, -0.15920539, 0.38349...","[-0.26172298, -1.4342418, 1.2562951, -1.448002...","[-0.071733244, -0.05542574, -0.16150494, -0.04..."
5,2000390617,14985,67256,12557,best ever banana cake with cream cheese frosting,0.952878,75.0,31.468750,1.937500,13.875000,0.937500,0.687500,1.562500,13,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0.004970295, -0.22123887, -0.20819537, 0.3866...","[1.5534351, 0.11940275, 1.3200809, 1.255067, 0...","[-0.27334324, -0.24490196, -0.14412951, -0.149..."
6,2000390617,14985,135350,23817,fannie farmer s classic baked macaroni cheese,0.952684,40.0,209.050000,20.000000,2.750000,9.250000,13.750000,5.250000,11,"[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.15615204, 0.3092702, -0.0536049, 0.3954, -0...","[1.5266387, 0.30980304, 1.5057268, 1.895238, 0...","[-0.014528584, 0.057631884, -0.19099052, -0.04..."
7,2000390617,14985,22782,3840,jo mama s world famous spaghetti,0.948781,80.0,55.590000,4.000000,4.500000,8.500000,5.900000,1.600000,9,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0.09439023, 0.2761577, -0.06896547, 0.358871,...","[1.1530571, 1.1041168, 1.859548, 2.0817826, 3....","[-0.23818788, -0.21035656, -0.18361913, -0.055..."
8,2000390617,14985,33919,6219,creamy burrito casserole,0.946127,50.0,64.075000,4.000000,2.000000,7.625000,5.375000,2.375000,4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.23837309, 0.19168904, -0.0969254, 0.496225...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.22582147, -0.21035656, -0.19414969, -0.066..."
9,2000390617,14985,50719,9417,the sweetest blueberry muffins,0.941046,30.0,21.366667,1.166667,6.916667,0.666667,0.666667,1.083333,13,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.048198022, 0.01900707, -0.066571094, 0.2700...","[2.3592212, 0.4622522, 1.5383724, 1.6835506, 0...","[-0.28806645, -0.25781286, -0.17343958, -0.152..."
